# Make bookkeeping with Holograms


- work with Weakly_2022_09
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/20


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [ ]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
collection = "u/dagoret/first_test_autxel"
datasetRefs1 = registry.queryDatasets(datasetType='postISRCCD', collections=collection, where= "instrument='LATISS'")
datasetRefs2 = registry.queryDatasets(datasetType='calexp', collections=collection, where= "instrument='LATISS'")

## List of Exposures

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs','group_name' ,'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','postISRCCD','calexp'])

In [ ]:
df_exposure.astype({"id":str,
                    "obs_id":str,
                    "day_obs": str,
                    "group_name":str,
                    "seq_num":'int32',
                    'type':str,
                    "target":str,
                    "filter":str,
                    "zenith_angle":'float',
                    "expos":'float',
                    "ra":'float',
                    "dec":'float',
                    "skyangle":'float',
                    "postISRCCD":bool,
                    "calexp":bool
                   }              
                  )

In [ ]:
#df_exposure = pd.DataFrame( {"id":'int64',
#                             "obs_id":str,
#                             "day_obs": str,
#                             "seq_num":'int32',
#                             "type":str,
#                             "target":str,
#                             "filter":str,
#                             "zenith_angle":'float32',
#                             "expos":'float32',
#                             "ra":'float32',
#                             "dec":'float32',
#                             "skyangle":'float32'})                  

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    flag_postisrccd = False
    flag_calexp = False
    
    for count, ref in enumerate(datasetRefs1):    
        if ref.dataId["exposure"]== info.id:
            flag_postisrccd = True
            break
    for count, ref in enumerate(datasetRefs2):    
        if ref.dataId["exposure"]== info.id:
            flag_calexp = True
            break        
            
    
    
    
    df_exposure.loc[count] = [str(info.id), str(info.obs_id), str(info.day_obs), str(info.group_name),int(info.seq_num), str(info.observation_type), str(info.target_name), str(info.physical_filter), info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,flag_postisrccd,flag_calexp]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t group_name:          ",info.group_name)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [ ]:
df_exposure.dtypes

In [ ]:
df_exposure

## Selection of science

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
df_science.tail(60)

# Generate Book Keeping files

In [ ]:
writer = pd.ExcelWriter('BookKeepingAuxtelRonchi_raw.xlsx', engine='xlsxwriter')
current_page = 0

In [ ]:
ListOfDates = df_science.day_obs.unique()

In [ ]:
ListOfDates

In [ ]:
for date_sel in ListOfDates:
    
    # get for that date
    df_science_selected = df_science[df_science.day_obs == date_sel ]
    N = len(df_science_selected)
    
    list_of_filters = df_science_selected['filter'].unique()
    
    selected_filters = []
    # Select hologram
    for filt in list_of_filters:
        #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
        #flag_sel = (filt.find('holo4') != -1) 
        flag_sel = (filt.find('ronchi170') != -1) 
        if flag_sel:
            selected_filters.append(filt) 
    Nfilt = len(selected_filters)
    # sort by filter name
    if(Nfilt>0):
        selected_filters = np.array(selected_filters)
        df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
        df_science_selected.to_excel(writer, sheet_name=str(date_sel))
        


In [ ]:
writer.save()